In [ ]:
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import plotly.graph_objects as go
import datetime
from sklearn.cluster import KMeans

# Introducao
## Bases Utilizadas

- [Base BNDES (Desembolsos)](https://dadosabertos.bndes.gov.br/dataset/desembolsos-mensais/resource/aac5c7bd-5d18-41c3-9720-52d20470427e) (Carregada por meio da API do portal de dados abertos do BNDES)

- [Base WDI (Países)](https://datatopics.worldbank.org/world-development-indicators/) (Carregada localmente)


# Base Desembolsos Mensais

In [ ]:
url_desembolsos = "https://dadosabertos.bndes.gov.br/dataset/c529f6a7-5ce4-4378-a585-92cbf53af710/resource/aac5c7bd-5d18-41c3-9720-52d20470427e/download/desembolsos-mensais.csv"

df_desembolsos = pd.read_csv(url_desembolsos, sep=';',decimal=",")

# Tratandos os dados dos desembolsos

In [ ]:
df_desembolsos['date'] = pd.to_datetime(df_desembolsos['ano'].astype(str) + '-' + df_desembolsos['mes'].astype(str) + '-' + "01")
df_desembolsos.loc[:,['regiao', 'uf', 'municipio']] = df_desembolsos.loc[:,['regiao', 'uf', 'municipio']].replace('\s*$', '',regex=True)
df_desembolsos.sort_values(by=['date'], inplace=True)
df_desembolsos.head()

# Agrupando os dados do BNDES por setor e subsetor e plotando

In [ ]:
desembolsosBySetor = pd.DataFrame(df_desembolsos.groupby(by=["ano", "setor_bndes"]).sum().desembolsos_reais).unstack()
desembolsosBySetor.columns = desembolsosBySetor.columns.levels[1]

sns.lineplot(data=desembolsosBySetor)
plt.savefig('../img/investimento_setor_bndes_bruto.svg')

In [ ]:
desembolsosBySubsetor = pd.DataFrame(df_desembolsos.groupby(by=["ano", "subsetor_bndes"]).sum().desembolsos_reais).unstack()
desembolsosBySubsetor = desembolsosBySubsetor.droplevel(0, axis=1)


fig = go.Figure([go.Scatter(x=desembolsosBySubsetor.index, y=desembolsosBySubsetor[i].values, name=i) for i in desembolsosBySubsetor.columns])
fig.show()

# Calculando a porcentagem de investimento em cada área e plotando

In [ ]:
porcentoSetor = desembolsosBySetor.div(desembolsosBySetor.sum(axis=1), axis=0)
porcentoSetor = porcentoSetor.apply(lambda x: x*100)

sns.lineplot(data=porcentoSetor)
plt.savefig('../img/investimento_setor_bndes_normalizado.svg')

In [ ]:
subsetores = ['AGROPECUÁRIA', 'ALIMENTO E BEBIDA']

porcentoSubsetor = desembolsosBySubsetor.div(desembolsosBySubsetor.sum(axis=1), axis=0)
porcentoSubsetor = porcentoSubsetor.apply(lambda x: x*100)

fig = go.Figure([go.Scatter(x=porcentoSubsetor.index, y=porcentoSubsetor[i].values, name=i) for i in subsetores])
fig.show()

# Base WDI

In [ ]:
wdi_zip = zipfile.ZipFile('../data/WDI_csv.zip')
df_wdi = pd.read_csv(wdi_zip.open('WDIData.csv'))

In [ ]:
df_wdi.head(3)

Vamos checar agora quais anos sao cobertos pelos dados do BNDES para filtrarmos a base WDI com dados de nosso interesse.

In [ ]:
lista_anos_interesse = df_desembolsos['ano'].sort_values().unique().astype(str)

print(lista_anos_interesse)

Vemos que sao dados a partir de 1995 com termino em 2020. Com essas informacoes, vamos filtrar as colunas da base wdi. Vamos manter as colunas contendo o nome do pais, codigo do pais e nome do indicador (Country Name, Country Code, Indicator Name)

In [ ]:
df_wdi[lista_anos_interesse]

In [ ]:
cols_wdi_indicadores =["Chemicals (% of value added in manufacturing)",
                        "Agriculture, forestry, and fishing, value added (% of GDP)",                        
                        "Industry (including construction), value added (% of GDP)",
                        "Services, value added (% of GDP)",
                        "Manufacturing, value added (% of GDP)",
                        "Food, beverages and tobacco (% of value added in manufacturing)",
                        "Textiles and clothing (% of value added in manufacturing)", 
                        "Machinery and transport equipment (% of value added in manufacturing)",
                        "Other manufacturing (% of value added in manufacturing)",
                        "GDP per capita growth (annual %)"]

In [ ]:
for i in cols_wdi_indicadores:
    print(i)

In [ ]:
for i in df_desembolsos['subsetor_bndes'].unique():
    print(i)

In [ ]:
cols_desembolsos_indicadores = ['QUÍMICA E PETROQUÍMICA',
                                'AGROPECUÁRIA',
                                'CONSTRUÇÃO',
                                'COMÉRCIO E SERVIÇOS',
                                'OUTROS TRANSPORTES',
                                'MATERIAL DE TRANSPORTE',
                                'ALIMENTO E BEBIDA',
                                'ENERGIA ELÉTRICA',
                                'CELULOSE E PAPEL',
                                'TELECOMUNICAÇÕES',
                                'METALURGIA E PRODUTOS',
                                'EXTRATIVA',
                                'TRANSPORTE FERROVIÁRIO',
                                'ATV. AUX. TRANSPORTES',
                                'MECÂNICA',                                
                                'SERV. UTILIDADE PÚBLICA',
                                'TÊXTIL E VESTUÁRIO',
                                'OUTRAS',
                                'TRANSPORTE RODOVIÁRIO',
                                'OUTROS']

In [ ]:
df_desembolsos['subsetor_cnae_agrupado'].sort_values().unique()

Selecionar melhor estrutura para o dicionario

In [ ]:
relacao_01 = {"quimica" : ["QUÍMICA E PETROQUÍMICA", "Chemicals (% of value added in manufacturing)"], 
              "agropecuaria": ["AGROPECUÁRIA", "Agriculture, forestry, and fishing, value added (% of GDP)"], 
              "industria": [["CELULOSE E PAPEL","METALURGIA E PRODUTOS","EXTRATIVA", "CONSTRUÇÃO"], "Industry (including construction), value added (% of GDP)"],
              "servicos":[["COMÉRCIO E SERVIÇOS", "ENERGIA ELÉTRICA", "TELECOMUNICAÇÕES", "TRANSPORTE FERROVIÁRIO", "ATV. AUX. TRANSPORTES", "SERV. UTILIDADE PÚBLICA", "TRANSPORTE RODOVIÁRIO"], "Services, value added (% of GDP)"],
              "maquinario":[["MATERIAL DE TRANSPORTE", "MECÂNICA"], "Machinery and transport equipment (% of value added in manufacturing)"],
              "alimentacao_tabaco": ["ALIMENTO E BEBIDA", "Food, beverages and tobacco (% of value added in manufacturing)"],
              "textil": ["TÊXTIL E VESTUÁRIO", "Textiles and clothing (% of value added in manufacturing)"] }

relacao_02 = {"Chemicals (% of value added in manufacturing)" : ["QUÍMICA E PETROQUÍMICA"], 
              "Agriculture, forestry, and fishing, value added (% of GDP)": ["AGROPECUÁRIA"], 
              "Industry (including construction), value added (% of GDP)": ["CELULOSE E PAPEL","METALURGIA E PRODUTOS","EXTRATIVA", "CONSTRUÇÃO"],
              "Services, value added (% of GDP)":[["COMÉRCIO E SERVIÇOS", "ENERGIA ELÉTRICA", "TELECOMUNICAÇÕES", "TRANSPORTE FERROVIÁRIO", "ATV. AUX. TRANSPORTES", "SERV. UTILIDADE PÚBLICA", "TRANSPORTE RODOVIÁRIO"]],
              "Machinery and transport equipment (% of value added in manufacturing)":["MATERIAL DE TRANSPORTE", "MECÂNICA"],
              "Food, beverages and tobacco (% of value added in manufacturing)": ["ALIMENTO E BEBIDA"],
              "Textiles and clothing (% of value added in manufacturing)": ["TÊXTIL E VESTUÁRIO"]}

# Agrupando os dados de desembolso de acordo com os setores definidos no dicionário relacao_01

In [ ]:
percentageSeries = [porcentoSubsetor[relacao_01[i][0]].T.sum() if isinstance(relacao_01[i][0], list) else porcentoSubsetor[relacao_01[i][0]] for i in relacao_01.keys()]

df_desembolsos_organizado = pd.DataFrame(percentageSeries, index=relacao_01.keys()).T
df_desembolsos_organizado.head()

# Tratando os dados da base WDI para usar-los na clusterização

In [ ]:
wdi_t = df_wdi.T
idx = pd.IndexSlice

wdi_t.columns = pd.MultiIndex.from_product([wdi_t.loc['Country Name'].unique(), wdi_t.loc['Indicator Name'].unique()])
wdi_t.drop(labels=['Country Name','Indicator Name', 'Country Code'], axis=0, inplace=True)
wdi_t.drop(index=['Indicator Code','Unnamed: 65'], inplace=True)
wdi_t.index = list(wdi_t.index.str.extract(r'([0-9]{4})').values.astype(int).reshape((1,-1)))
wdi_t.index = [i[0] for i in wdi_t.index.to_flat_index()]
wdi_t.index = wdi_t.index.astype(int)
wdi_t.replace(regex=r'^.*$', value=np.nan, inplace=True)
wdi_t = wdi_t.apply(pd.to_numeric)
wdi_t.head()

In [ ]:
countries = ['Brazil', 'United States', 'China', 'Germany', 'United Kingdom', 'Japan']

sns.lineplot(data=wdi_t.loc[lista_anos_interesse.astype(int),idx[countries, ['GDP (current US$)']]])
plt.savefig('../img/pib_paises.svg')

In [ ]:
countries = ['Brazil']

sns.heatmap(
    wdi_t.loc[lista_anos_interesse.astype(int),idx[countries,[relacao_01['servicos'][1]] + [relacao_01['industria'][1]] + [relacao_01['agropecuaria'][1]] + ['GDP growth (annual %)']]].corr(),
    annot=True,
    linewidths=.5,
    xticklabels=['% de investimento em Serviços', '% de investimento em Indústria', '% de investimento em Agropecuária', 'PIB'], 
    yticklabels=['% de investimento em Serviços', '% de investimento em Indústria', '% de investimento em Agropecuária', 'PIB']
)
plt.savefig('../img/corr.svg')

In [ ]:
setor='industria'
countries = ['Brazil', 'Spain', 'Mexico', 'Turkey', 'Germany','World']

fig = go.Figure([go.Scatter(
    x=wdi_t.loc[:,idx[:,relacao_01[setor][1]]].index, 
    y=wdi_t.loc[:,idx[:,relacao_01[setor][1]]][i].values.flatten(), name=i) 
    for i in countries])
fig.show()

# Construindo o dataframe concatenando os dados da wdi e do BNDES para aplcar a clusterização

In [ ]:
setor = "industria"

aux_wdi = wdi_t[4:].loc[:,pd.IndexSlice[:, relacao_01[setor][1]]]
aux_wdi = aux_wdi.droplevel(1, axis=1)
aux_wdi = aux_wdi.fillna(0)
aux_wdi = aux_wdi.apply(lambda x: x.replace(0, x.iloc[np.array(x).nonzero()].mean()) if len(np.array(x).nonzero()[0]) > 0 else x
                        , axis=0)
aux_wdi = aux_wdi.loc[lista_anos_interesse.astype(int)]

concat = pd.concat([aux_wdi, pd.DataFrame({'BNDES': df_desembolsos_organizado[setor]})], axis=1)
concat.head()

In [ ]:
url_balanco = "https://dadosabertos.bndes.gov.br/dataset/8d01af82-301b-4481-a2fc-3daf3a7bdf03/resource/159c6303-973d-413a-8de7-5343c318d37e/download/indicadores-financeiros-balanco-patrimonial-serie-historica.csv"
df_balanco = pd.read_csv(url_balanco, sep=';',decimal=",")
df_balanco['data'] = pd.to_datetime(df_balanco['data'])
df_balanco['ano'] = df_balanco.data.dt.year
df_balanco_interesse = df_balanco[["patrimonio_liquido", 'ano']].groupby('ano').mean()
pct_change = df_balanco_interesse.pct_change()
pct_change = pct_change * 100

In [ ]:
paises = ['Brazil', 'BNDES', 'patrimonio_liquido', 'PIB Brasileiro']

concat_gdp = pd.concat([concat, pct_change, wdi_t.loc[lista_anos_interesse.astype(int),idx[['Brazil'], ['GDP growth (annual %)']]]], axis=1)
col = list(concat_gdp.columns[:-1])
col.append('PIB Brasileiro')
concat_gdp.columns = col

sns.lineplot(data=concat_gdp.loc[lista_anos_interesse.astype(int), paises])
plt.savefig('../img/brasil_bndes_crescimentopib.svg')

In [ ]:
sns.heatmap(concat_gdp.loc[lista_anos_interesse.astype(int), paises].corr(),
    annot=True,
    linewidths=.5)

# Clusterizando

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, pais='BNDES'):
    pais_index = list(concat.columns).index(pais) 
    
    clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat.values.T)
    lista_paises = concat.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(10,30))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.show()

In [ ]:
countries = np.array(paises_similares_dict['25'])

fig = go.Figure([go.Scatter(
    x=concat.index, 
    y=concat[i], name=i) 
    for i in countries])
fig.show()

# Construindo o dataframe do crescimento percentual por anos nos investimentos por subsetor do BNDES

In [ ]:
pct_change_subsetor = desembolsosBySubsetor.pct_change()
pct_change_subsetor.head()

# Construindo o dataframe do crescimento percentual por anos nos investimentos por setor do BNDES

In [ ]:
pct_change_setor = pd.DataFrame(df_desembolsos.groupby(by=["ano", "setor_bndes"]).sum().desembolsos_reais).unstack()
pct_change_setor.columns = pct_change_setor.columns.levels[1]
pct_change_setor = pct_change_setor.pct_change()
pct_change_setor.head()

In [ ]:
wdi_annual_growth = df_wdi[df_wdi['Indicator Name'].str.match(r'^.*annual % growth.*$')].T
idx = pd.IndexSlice

wdi_annual_growth.columns = pd.MultiIndex.from_product([wdi_annual_growth.loc['Country Name'].unique(), wdi_annual_growth.loc['Indicator Name'].unique()])
wdi_annual_growth.drop(labels=['Country Name','Indicator Name', 'Country Code'], axis=0, inplace=True)
wdi_annual_growth.drop(index=['Indicator Code','Unnamed: 65'], inplace=True)
wdi_annual_growth.index = list(wdi_annual_growth.index.str.extract(r'([0-9]{4})').values.astype(int).reshape((1,-1)))
wdi_annual_growth.replace(regex=r'^.*$', value=np.nan, inplace=True)
wdi_annual_growth = wdi_annual_growth.apply(pd.to_numeric)
wdi_annual_growth.head()

In [ ]:
relacao_pct_change = { "agropecuaria": ["AGROPECUÁRIA", "Agriculture, forestry, and fishing, value added (annual % growth)"], 
              "industria": [ "INDUSTRIA", "Industry (including construction), value added (annual % growth)"],
              "servicos": ["COMÉRCIO E SERVIÇOS", "Services, value added (annual % growth)"] }

In [ ]:
setor = "servicos"

aux_wdi_pct_change = wdi_annual_growth[4:].loc[:,pd.IndexSlice[:, relacao_pct_change[setor][1]]]
aux_wdi_pct_change = aux_wdi_pct_change.droplevel(1, axis=1)
aux_wdi_pct_change = aux_wdi_pct_change.fillna(0)
aux_wdi_pct_change = aux_wdi_pct_change.apply(lambda x: x.replace(0, x.iloc[np.array(x).nonzero()].mean()) if len(np.array(x).nonzero()[0]) > 0 else x
                        , axis=0)
aux_wdi_pct_change.index = [i[0] for i in aux_wdi_pct_change.index.to_flat_index()]
aux_wdi_pct_change.index = aux_wdi_pct_change.index.astype(int)
aux_wdi_pct_change = aux_wdi_pct_change.loc[lista_anos_interesse[1:].astype(int)]

concat_pct_change = pd.concat([aux_wdi_pct_change, pd.DataFrame({'BNDES': pct_change_setor.loc[1996:, relacao_pct_change[setor][0]]})], axis=1)
concat_pct_change.head()

# Clusterizando para mudança percentual

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, pais='BNDES'):
    pais_index = list(concat_pct_change.columns).index(pais) 
    
    clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat_pct_change.values.T)
    lista_paises = concat_pct_change.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(40,50))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.show()

In [ ]:
countries = np.array(paises_similares_dict['40'])

fig = go.Figure([go.Scatter(
    x=concat.index, 
    y=concat[i], name=i) 
    for i in countries])
fig.show()